In [1]:
import pandas as pd
import re
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import NMF, TruncatedSVD
from gensim import corpora, models, matutils

In [9]:
df = pd.read_pickle('../data/CannesFilmFestival_tweets_final')
tweets=df[df['language']=='en']['tweet']

In [4]:
print(tweets.iloc[6])

#RebeccaHall in #Gucci at the75th #CannesFilmFestival #FinalCut #premiere &amp; Opening Ceremony. More on the blog, link in bio. #tapforcredits #Cannes #fashion #celebrityfashion #fashionblogger #TheFashionistaStories  https://t.co/OKuWuJjK96


In [10]:
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub(r"[^\w\d'\s]+",'',temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [11]:
stopwords= ['cannes','cannesfilm','cannesfilmfestival','cannesfilmfestival','festival','film',
            'annual','cannesfilmfestival2022','cannes2022','cannes75','2022','75th']

In [12]:
cleaned_tweet = [clean_tweet (tw)for tw in tweets]

/var/folders/s5/kqm0k4xj165fxvtjxq0njrq00000gn/T/ipykernel_40567/3763913734.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:


In [13]:
len(cleaned_tweet)

29185

In [14]:
label = [e[:100]+"..." for e in cleaned_tweet]

In [55]:
### BEGIN SOLUTION
cv = CountVectorizer(stop_words='english')#, min_df=.01)
### END SOLUTION
cv

CountVectorizer(stop_words='english')

In [62]:
tweet_cv = cv.fit_transform(cleaned_tweet)

In [66]:
dtm_cv = pd.DataFrame(tweet_cv.toarray(),index=label, columns=cv.get_feature_names())
### END SOLUTION
dtm_cv

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000am,001,0045,007,028,0400,0526,0527,...,zunino,zurich,zurzolo,zuza,zwei,zxai,zyaada,zylberstein,zymvragaki,zzz
cant wait for see actress lee jieun broker brokerteam brokeratcannes2022 leejieun cannes2022 iu...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
paranjithincannes cannesfilmfestival2022 coat suit poduvan da style ah getha...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cant wait leejieun iu at cannes2022 with broker team...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
thank you for this article we are very eager to see leejieun performance as soyoung in koreeda movie...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
broker next iu leejieun brokeratcannes2022 cannes2022...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oh my girl cannesfilmfestival2022...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
who are the sexiest dressed stars at cannes over the years read more from our new articles cannesfil...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
top gun maverick cannes2022 film festival photocall amp premiere...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
my vlog from day three of cannes cannes2022 cannes75 is up here with my favourite of the competition...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
### BEGIN SOLUTION
tv = TfidfVectorizer(stop_words='english')
tweet_tv = tv.fit_transform(cleaned_tweet)
tfidf = pd.DataFrame(tweet_tv.toarray(), index=label,columns=tv.get_feature_names())
### END SOLUTION
tfidf

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000am,001,0045,007,028,0400,041,0526,...,zurich,zurichfilmfestival,zurzolo,zuza,zwei,zxai,zyaada,zylberstein,zymvragaki,zzz
cant wait for see actress lee jieun broker brokerteam brokeratcannes2022 leejieun iu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
paranjithincannes coat suit poduvan da style ah getha...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cant wait leejieun iu at with broker team...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
thank you for this article we are very eager to see leejieun performance as soyoung in koreeda movie...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
broker next iu leejieun brokeratcannes2022...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kylie and her remix classicsremixed...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
festivaldecannes super model adriana lima very sweet actress wilma from turkey and kavas 20220520 ac...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
we at kothare vision are pleased to announce our new vision storytellers nook pvt ltd a company that...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
revisit these unforgettable 90s moments from the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fit NMF Model with Count Vectorizor  

In [ ]:
nmf= NMF(10, init='nndsvda')
nmf.fit(dtm_cv)
topic_term = nmf.components_
topic_term_df = pd.DataFrame (topic_term.round(3), columns = cv.get_feature_names())

In [20]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [78]:
output =display_topics (nmf, cv.get_feature_names(),10)
output;
#Doesn't look like it worked too well, lot of hardcore fans flooding certain celebrity names


Topic  1
cannes2022, hinakhan, review, like, just, future, look, love, new, time

Topic  2
cannes, 2022, festival, france, khan, gala, amfar, redcarpet, hina, cannes75

Topic  3
cannesfilmfestival, premiere, fashion, 75th, india, indiaatcannes, link, dress, blog, gown

Topic  4
cannesfilmfestival2022, hinakhan, hinakhanatcannes2022, hina, look, looking, hellyshah, looks, queen, cannesredcarpet

Topic  5
film, festival, director, joyland, new, du, premiere, 75th, india, industry

Topic  6
red, carpet, gown, walked, looks, walk, premiere, day, festival, redcarpet

Topic  7
broker, iu, leejieun, lee, brokercannes2022, team, actress, jieun, ji, eun

Topic  8
75th, screening, annual, attends, elvis, festival, anniversary, france, celebration, attending

Topic  9
amp, premiere, india, world, films, indiaatcannes, director, like, actors, movie

Topic  10
deepikapadukone, deepikaatcannes, deepika, padukone, look, deepikaascannesjury, gown, black, day, cannes2022




/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fit NMF Model with TFIDF

In [32]:
nmf_tfidf= NMF(15, init='nndsvda')
nmf_tfidf.fit(tfidf)
topic_term = nmf_tfidf.components_
topic_term_df = pd.DataFrame (topic_term.round(3), columns = tv.get_feature_names())

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
output =display_topics (nmf_tfidf, tv.get_feature_names(),10) # inspect top 10 terms of each topic
output;
# top movies: Broker, Armageddon Time, Crimes of the Future, Elvis, Decision to Leave, Triangle of Sadness
# top celebrities: IU, Deepika Padukone, Hina Khan, Anne Hathaway, Song Kangho, Helly Shah,Kristin Stewart,
#                   Bella Hadid, Austin butler, Aishwarya Rai


Topic  1
broker, iu, leejieun, lee, brokercannes2022, team, bonvoyagebrokerteam, brokeratcannes2022, ji, eun

Topic  2
deepikapadukone, deepikaatcannes, deepikaascannesjury, deepika, padukone, gown, louisvuitton, gorgeous, instagram, vuitton

Topic  3
hinakhan, hinakhanatcannes2022, hina, khan, aced, hinakhanatcannes, looks, looking, stunning, princess

Topic  4
screening, attends, time, armageddon, linnocent, anniversary, celebration, innocent, frere, petit

Topic  5
red, carpet, gown, ceremony, closing, premiere, walked, looks, walk, lee

Topic  6
anne, hathaway, annehathaway, armageddon, time, jeremy, armageddontime, strong, photocall, greg

Topic  7
best, actor, song, actress, kangho, award, kang, director, ho, wins

Topic  8
hellyshah, hellyshahatcannes, kayapalat, hellyshahatcannes2022, hellyaskaya, hellyholics, helly, poster, hellykelog, shah

Topic  9
kristen, stewart, crimes, future, crimesofthefuture, kristenstewart, photocall, david, premiere, cronenberg

Topic  10
bella, h

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Try LSA with TFIDF

In [82]:
lsa = TruncatedSVD(10)
lsa.fit(tfidf)
lsa

TruncatedSVD(n_components=10)

In [84]:
topic_term= lsa.components_.round(3)
topic_term

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., -0.,  0.,  0.],
       [ 0.,  0., -0., ..., -0.,  0., -0.],
       ...,
       [-0., -0.,  0., ...,  0., -0.,  0.],
       [-0.,  0.,  0., ..., -0., -0.,  0.],
       [-0., -0.,  0., ...,  0., -0., -0.]])

In [85]:
### BEGIN SOLUTION
topic_term_df = pd.DataFrame(topic_term.round(3),
                columns = tv.get_feature_names())
### END SOLUTION
topic_term_df

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000am,001,0045,007,028,0400,0526,0527,...,zunino,zurich,zurzolo,zuza,zwei,zxai,zyaada,zylberstein,zymvragaki,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,...,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.000,0.0,0.000,...,-0.000,0.0,-0.000,0.0,-0.0,-0.000,-0.0,-0.000,0.0,0.0
2,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.000,-0.0,-0.001,...,-0.000,0.0,-0.000,0.0,-0.0,-0.000,-0.0,-0.000,0.0,-0.0
3,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.000,0.0,-0.000,...,0.000,-0.0,-0.000,-0.0,-0.0,0.000,-0.0,0.000,-0.0,0.0
4,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.000,-0.0,-0.000,...,-0.000,-0.0,-0.001,-0.0,-0.0,0.000,0.0,0.000,-0.0,0.0
5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.000,0.0,0.000,...,0.001,-0.0,-0.001,-0.0,0.0,-0.001,0.0,-0.000,-0.0,-0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.000,0.0,-0.000,...,0.000,0.0,0.000,0.0,-0.0,0.001,-0.0,0.001,0.0,0.0
7,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.001,0.0,0.000,...,0.000,-0.0,0.001,-0.0,-0.0,0.001,-0.0,0.000,-0.0,0.0
8,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.000,0.0,-0.000,...,0.001,-0.0,0.000,-0.0,-0.0,0.000,0.0,-0.000,-0.0,0.0
9,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,-0.000,-0.0,-0.000,...,0.000,-0.0,-0.000,-0.0,-0.0,-0.000,-0.0,0.000,-0.0,-0.0


In [88]:
output=display_topics(lsa,tv.get_feature_names(),10)
output
# NMF appears to work better


Topic  1
cannes, cannes2022, cannesfilmfestival2022, cannesfilmfestival, film, festival, red, 2022, deepikapadukone, carpet

Topic  2
broker, iu, leejieun, brokercannes2022, cannes2022, lee, bonvoyagebrokerteam, team, brokeratcannes2022, actress

Topic  3
hinakhan, cannesfilmfestival2022, hinakhanatcannes2022, deepikapadukone, hina, look, deepikaatcannes, queen, red, carpet

Topic  4
cannesfilmfestival, deepikapadukone, red, carpet, deepikaatcannes, deepika, deepikaascannesjury, padukone, gown, day

Topic  5
cannesfilmfestival, cannes, 2022, hinakhan, iu, broker, leejieun, fashion, hinakhanatcannes2022, bella

Topic  6
red, carpet, film, festival, cannes, 2022, walked, lee, walk, hina

Topic  7
cannesfilmfestival, kristen, stewart, cannes2022, future, crimes, carpet, premiere, crimesofthefuture, red

Topic  8
screening, hinakhan, elvis, attends, red, carpet, 75th, hinakhanatcannes2022, annual, cannesfilmfestival

Topic  9
hellyshah, anne, hellyshahatcannes, hathaway, cannesfilmfestiva

(TruncatedSVD(n_components=10),
 ['00',
  '000',
  '000am',
  '001',
  '0045',
  '007',
  '028',
  '0400',
  '0526',
  '0527',
  '0830am',
  '09',
  '0930',
  '10',
  '100',
  '1000',
  '10000',
  '100am',
  '100carat',
  '100s',
  '100th',
  '100x',
  '101',
  '1010',
  '10100',
  '1025',
  '1030',
  '1030183',
  '1030pm',
  '105',
  '10715carat',
  '10b',
  '10k',
  '10min',
  '10mins',
  '10minute',
  '10minutelong',
  '10minutes',
  '10pm',
  '10th',
  '10yearsold',
  '10yrs',
  '11',
  '110',
  '1100',
  '111',
  '1111',
  '112',
  '1120',
  '1125',
  '1130',
  '1130am',
  '114',
  '1145am',
  '118',
  '119',
  '11am',
  '11th',
  '11yearold',
  '12',
  '120',
  '1200',
  '1200p',
  '120473',
  '121',
  '1225',
  '1230',
  '1230130',
  '12306',
  '1230am',
  '1239',
  '123telugu',
  '124489',
  '125',
  '125000',
  '126',
  '129',
  '12am',
  '12min',
  '12minute',
  '12minutelong',
  '12minuteovation',
  '12minutesstandingovation',
  '12minutestandingovation',
  '12pm',
  '12th',

Try LDA

In [18]:
term_doc = tweet_tv.transpose()
corpus = matutils.Sparse2Corpus(term_doc)
id2word = dict((v, k) for k, v in tv.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=10, id2word=id2word, passes=10)
lda

In [ ]:
lda.print_topics()

Use Chosen model of NMF with TFIDF

In [43]:
output =display_topics (nmf_tfidf, tv.get_feature_names(),10) # inspect top 10 terms of each topic
output;


Topic  1
broker, iu, leejieun, lee, brokercannes2022, team, bonvoyagebrokerteam, brokeratcannes2022, ji, eun

Topic  2
deepikapadukone, deepikaatcannes, deepikaascannesjury, deepika, padukone, gown, louisvuitton, gorgeous, instagram, vuitton

Topic  3
hinakhan, hinakhanatcannes2022, hina, khan, aced, hinakhanatcannes, looks, looking, stunning, princess

Topic  4
screening, attends, time, armageddon, linnocent, anniversary, celebration, innocent, frere, petit

Topic  5
red, carpet, gown, ceremony, closing, premiere, walked, looks, walk, lee

Topic  6
anne, hathaway, annehathaway, armageddon, time, jeremy, armageddontime, strong, photocall, greg

Topic  7
best, actor, song, actress, kangho, award, kang, director, ho, wins

Topic  8
hellyshah, hellyshahatcannes, kayapalat, hellyshahatcannes2022, hellyaskaya, hellyholics, helly, poster, hellykelog, shah

Topic  9
kristen, stewart, crimes, future, crimesofthefuture, kristenstewart, photocall, david, premiere, cronenberg

Topic  10
bella, h

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [50]:
topics =['Broker', 'Deepika Padukone','Hina Khan', 'French movies','Red Carpet','Armageddon Time',
'Kang Song Ho','Helly Shah','Crimes of the Future','Bella Hadid','Elvis','Decision to Leave'
,'Triangle of Sadness','Aishwarya Rai','Aishwarya Rai']

display_topics(nmf_tfidf, tv.get_feature_names(), 10, topics
);


Topic:  Broker
broker, iu, leejieun, lee, brokercannes2022, team, bonvoyagebrokerteam, brokeratcannes2022, ji, eun

Topic:  Deepika Padukone
deepikapadukone, deepikaatcannes, deepikaascannesjury, deepika, padukone, gown, louisvuitton, gorgeous, instagram, vuitton

Topic:  Hina Khan
hinakhan, hinakhanatcannes2022, hina, khan, aced, hinakhanatcannes, looks, looking, stunning, princess

Topic:  French movies
screening, attends, time, armageddon, linnocent, anniversary, celebration, innocent, frere, petit

Topic:  Red Carpet
red, carpet, gown, ceremony, closing, premiere, walked, looks, walk, lee

Topic:  Armageddon Time
anne, hathaway, annehathaway, armageddon, time, jeremy, armageddontime, strong, photocall, greg

Topic:  Kang Song Ho
best, actor, song, actress, kangho, award, kang, director, ho, wins

Topic:  Helly Shah
hellyshah, hellyshahatcannes, kayapalat, hellyshahatcannes2022, hellyaskaya, hellyholics, helly, poster, hellykelog, shah

Topic:  Crimes of the Future
kristen, stewart

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [48]:
doc_topic = nmf_tfidf.transform(tweet_tv)
doc_topic.shape

/Users/amykim/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  warnings.warn(


(29185, 15)

In [51]:
doc_topic_df = pd.DataFrame(doc_topic.round(5), index=label, columns=topics)
doc_topic_df

,Broker,Deepika Padukone,Hina Khan,French movies,Red Carpet,Armageddon Time,Kang Song Ho,Helly Shah,Crimes of the Future,Bella Hadid,Elvis,Decision to Leave,Triangle of Sadness,Aishwarya Rai,Aishwarya Rai
cant wait for see actress lee jieun broker brokerteam brokeratcannes2022 leejieun iu...,0.09272,0.00000,0.00000,0.00000,0.00307,0.00000,0.01469,0.00149,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
paranjithincannes coat suit poduvan da style ah getha...,0.00000,0.00077,0.00105,0.00000,0.00067,0.00036,0.00080,0.00025,0.00067,0.00062,0.00000,0.00000,0.00000,0.00275,0.00081
cant wait leejieun iu at with broker team...,0.11841,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00531,0.00000,0.00000,0.00000,0.00000,0.00384,0.00000,0.00000
thank you for this article we are very eager to see leejieun performance as soyoung in koreeda movie...,0.04728,0.00000,0.00000,0.00000,0.00000,0.00000,0.00469,0.00066,0.00000,0.00000,0.00166,0.00109,0.00235,0.00000,0.00000
broker next iu leejieun brokeratcannes2022...,0.13412,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kylie and her remix classicsremixed...,0.00000,0.00000,0.00000,0.00000,0.00059,0.00000,0.00000,0.00000,0.00000,0.00000,0.00294,0.00000,0.00000,0.00023,0.00000
festivaldecannes super model adriana lima very sweet actress wilma from turkey and kavas 20220520 ac...,0.00191,0.00000,0.00061,0.00190,0.00185,0.00000,0.01573,0.00010,0.00000,0.00050,0.00059,0.00000,0.00000,0.00244,0.00001
we at kothare vision are pleased to announce our new vision storytellers nook pvt ltd a company that...,0.00000,0.00044,0.00045,0.00007,0.00035,0.00028,0.00000,0.00049,0.00049,0.00000,0.00052,0.00216,0.00288,0.00551,0.00000
revisit these unforgettable 90s moments from the...,0.00000,0.00000,0.00000,0.00000,0.00092,0.00016,0.00340,0.00028,0.00000,0.00003,0.00004,0.00000,0.00000,0.00006,0.00022
